<a href="https://colab.research.google.com/github/MoistMaster64/UFV-Visualizacion/blob/main/PR1_Visualizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [113]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/datos_ejercicio_ventas.csv')
df.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


In [114]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18666 entries, 0 to 18665
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   COUNTRY        18666 non-null  object 
 1   SUBBRAND       18666 non-null  object 
 2   YEAR           18666 non-null  int64  
 3   MONTH          18666 non-null  int64  
 4   SCENARIO       18666 non-null  object 
 5   FORECAST       17766 non-null  object 
 6   FORECAST_YEAR  17766 non-null  float64
 7   AMOUNT         18666 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.1+ MB


In [74]:
#Los valores nulos son de los actuals, que no tienen forecast

In [115]:
df.describe()


,YEAR,MONTH,FORECAST_YEAR,AMOUNT
count,18666.000000,18666.000000,17766.0,1.866600e+04
mean,2023.716383,6.475463,2023.0,9.721822e+05
std,0.590782,3.463632,0.0,1.915283e+06
min,2023.000000,1.000000,2023.0,-2.171201e+05
25%,2023.000000,3.000000,2023.0,8.754541e+04
50%,2024.000000,6.000000,2023.0,3.081759e+05
75%,2024.000000,9.000000,2023.0,1.078576e+06
max,2025.000000,12.000000,2023.0,1.481563e+07


1) Distribución de ventas:

    a) Por país
    

In [116]:
#Se asume que la distribución de ventas se refiere a las actuales y no a las predicciones.
df_actuals = df[df['SCENARIO'] == 'actual'].reset_index()
df_forecast = df[df['SCENARIO'] == 'AI_forecast'].reset_index()
df_forecast.head()

,index,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


In [117]:
sales_by_country = df_actuals.groupby('COUNTRY')['AMOUNT'].sum().reset_index()
print(sales_by_country)

         COUNTRY        AMOUNT
0          Czech  3.535164e+07
1        Denmark  5.659668e+07
2  Great Britain  3.347786e+08
3        Hungary  4.153991e+07
4          Italy  4.345404e+07
5    Netherlands  6.395943e+07
6         Norway  5.121406e+07
7       Portugal  3.488807e+07
8          Spain  8.131266e+06


In [118]:
import plotly.express as px
#Hacemos un grafico de barras mostrando ventas por país de menor a mayor
sales_by_country_sorted = sales_by_country.sort_values(by=['AMOUNT'])

# Create a Plotly bar chart
fig = px.bar(sales_by_country_sorted,
             x=sales_by_country_sorted.COUNTRY,
             y='AMOUNT',
             title='Ventas Totales por País (de Menor a Mayor)',
             labels={'x': 'País', 'AMOUNT': 'Numero de Ventas'})

# Definimos un layout más específico
fig.update_layout(
    title={'text': 'Ventas Totales por País (de Menor a Mayor)', 'x': 0.5},  # Centramos textos
    xaxis_title="País",
    yaxis_title="Número de Ventas",
    xaxis=dict(categoryorder='total ascending')
)

fig.show()


  b) Por mes y año

In [119]:
#Para que solo sea sobre actuals:
df = df_actuals
# Creamos nueva columna que junte el mes y el año "AAAA-MM"
df['YEAR_MONTH'] = df['YEAR'].astype(str) + '-' + df['MONTH'].astype(str).str.zfill(2)

# Group by this new YEAR_MONTH column and sum the sales amount
sales_by_month = df.groupby('YEAR_MONTH')['AMOUNT'].sum().sort_index()

# Creamos un diagrama de lineas ya que hay muchos meses y tiene sentido que se vean secuencialmente
fig = px.line(sales_by_month,
              x=sales_by_month.index,
              y='AMOUNT',
              title='Ventas Totales por Mes y Año',
              labels={'x': 'Mes y Año', 'AMOUNT': 'Número de Ventas'})

# Definimos un layout más específico
fig.update_layout(
    title={'text': 'Ventas Totales por Mes y Año', 'x': 0.5},
    xaxis_title="Mes y Año",
    yaxis_title="Número de Ventas",
    xaxis_tickangle=-45  # Rotamos las etiquetas del eje x para facilitar la lectura de los meses
)

fig.show()


  c) Distribución por Submarca

In [120]:
# Calculate total sales by SUBBRAND
sales_by_subbrand = df.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()

# Sort sales by SUBBRAND from lowest to highest
sales_by_subbrand_sorted = sales_by_subbrand.sort_values(by='AMOUNT')

# Create the Plotly bar chart
fig = px.bar(sales_by_subbrand_sorted,
             x='SUBBRAND',  # Use SUBBRAND column for x-axis
             y='AMOUNT',
             title='Ventas Totales por Submarca (de Menor a Mayor)',
             labels={'SUBBRAND': 'Submarca', 'AMOUNT': 'Numero de Ventas'})

# Customize layout
fig.update_layout(
    title={'text': 'Ventas Totales por Submarca (de Menor a Mayor)', 'x': 0.5},
    xaxis_title="Submarca",
    yaxis_title="Número de Ventas"
)

fig.show()

In [121]:
# Create the Plotly pie chart
fig = px.pie(sales_by_subbrand,
             values='AMOUNT',
             names='SUBBRAND',
             title='Distribución de Ventas por Submarca')

# Customize layout (optional)
fig.update_layout(
    title={'text': 'Distribución de Ventas por Submarca', 'x': 0.5}  # título centrado
)

fig.show()

2) Cual es la tendencia y estacionalidad de:
    a) Todas las ventas del país con menos ventas.

In [123]:
# Filter data for Spain
df_spain = df[df['COUNTRY'] == 'Spain']

# Group by YEAR_MONTH and sum the sales amount, then sort by YEAR_MONTH
sales_by_month = df_spain.groupby('YEAR_MONTH')['AMOUNT'].sum().sort_index()

# Create line chart with all layout options in one step
fig = px.line(
    sales_by_month,
    x=sales_by_month.index,
    y=sales_by_month.values,
    title='Ventas Totales por Mes y Año en España',
    labels={'x': 'Mes y Año', 'y': 'Número de Ventas'},
    template='plotly_white'
)

# Define layout
fig.update_layout(
    title={'x': 0.5},  # Center the title
    xaxis_title='Mes y Año',
    yaxis_title='Número de Ventas',
    xaxis_tickangle=-45  # Rotate x-axis labels for readability
)

# Display the figure
fig.show()

In [124]:
# Agrupamos los datos por el mes y sumamos la cantidad de ventas
monthly_sales = df_spain.groupby('MONTH')['AMOUNT'].sum()

# Creamos un gráfico de línea para las ventas mensuales agregadas
fig = px.line(
    monthly_sales,
    x=monthly_sales.index,
    y=monthly_sales.values,
    title='Ventas Totales por Mes (Análisis Estacional)',
    labels={'x': 'Mes', 'y': 'Número de Ventas'},
    template='plotly_white'
)

# Definimos el layout para mejorar la legibilidad del gráfico
fig.update_layout(
    title={'x': 0.5},
    xaxis_title='Mes',
    yaxis_title='Número de Ventas',
    xaxis_tickmode='array',
    xaxis_tickvals=list(range(1, 13)),
    xaxis_ticktext=['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
)

# Mostramos la figura
fig.show()


b) Todas las ventas de la marca con más ventas (Pepsi max)

In [125]:
# Filtramos los datos para el subbrand 'Pepsi Max'
df_pepsimax = df[df['SUBBRAND'] == 'Pepsi Max (L3)']

# Agrupamos las ventas por mes y año, sumando las ventas de cada mes
sales_over_time = df_pepsimax.groupby('YEAR_MONTH')['AMOUNT'].sum()

# Graficamos la evolución de las ventas a lo largo del tiempo
fig1 = px.line(
    sales_over_time,
    x=sales_over_time.index,
    y=sales_over_time.values,
    title='Evolución de Ventas de Pepsi Max por Mes y Año',
    labels={'x': 'Mes y Año', 'y': 'Número de Ventas'},
    template='plotly_white'
)

fig1.update_layout(
    title={'x': 0.5},
    xaxis_title='Mes y Año',
    yaxis_title='Número de Ventas',
    xaxis_tickangle=-45
)

fig1.show()

In [126]:
monthly_sales = df_pepsimax.groupby('MONTH')['AMOUNT'].sum()

# Graficamos la estacionalidad (ventas mensuales agregadas)
fig2 = px.line(
    monthly_sales,
    x=monthly_sales.index,
    y=monthly_sales.values,
    title='Ventas Totales Mensuales de Pepsi Max (Análisis Estacional)',
    labels={'x': 'Mes', 'y': 'Número de Ventas'},
    template='plotly_white'
)

fig2.update_layout(
    title={'x': 0.5},
    xaxis_title='Mes',
    yaxis_title='Número de Ventas',
    xaxis_tickmode='array',
    xaxis_tickvals=list(range(1, 13)),
    xaxis_ticktext=['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
)

fig2.show()

In [127]:
# Creamos un diccionario que mapea los códigos de pronóstico a los números de mes correspondientes
forecast_map = {
    'AI_P02F': 1, 'AI_P03F': 2, 'AI_P04F': 3, 'AI_P05F': 4,
    'AI_P06F': 5, 'AI_P07F': 6, 'AI_P08F': 7, 'AI_P09F': 8,
    'AI_P10F': 9, 'AI_P11F': 10, 'AI_P12F': 11, 'AI_PF': 12
}

# Añadimos una nueva columna al DataFrame para el mes del pronóstico,
# utilizando el diccionario para mapear la columna 'FORECAST' al mes correspondiente
df_forecast['FORECAST_MONTH'] = df_forecast['FORECAST'].map(forecast_map)

# Definimos una función para calcular la diferencia en meses entre el mes del pronóstico y la fecha de los datos
def calculate_month_diff(row):
    forecast_year = row['FORECAST_YEAR']  # Año del pronóstico
    forecast_month = row['FORECAST_MONTH']  # Mes del pronóstico
    data_year = row['YEAR']  # Año de los datos reales
    data_month = row['MONTH']  # Mes de los datos reales

    # Calculamos los meses totales para ambas fechas (pronóstico y datos)
    forecast_total_months = forecast_year * 12 + forecast_month  # Total de meses para el pronóstico
    data_total_months = data_year * 12 + data_month  # Total de meses para los datos

    # Calculamos la diferencia en meses entre el pronóstico y los datos
    return data_total_months - forecast_total_months

# Aplicamos la función para calcular la diferencia en meses para cada fila del DataFrame
df_forecast['MONTH_DIFF'] = df_forecast.apply(calculate_month_diff, axis=1)


In [128]:
df_forecast.head()

,index,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT,FORECAST_MONTH,MONTH_DIFF
0,0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194,1,11.0
1,1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029,9,3.0
2,2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847,8,4.0
3,3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516,9,15.0
4,4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491,2,7.0


In [143]:
# Filter data for Spain

df_spain_actuals = df_actuals[df_actuals['COUNTRY'] == 'Spain']
df_spain_forecast = df_forecast[df_forecast['COUNTRY'] == 'Spain']
#añadimos la columna Month_year al df de forecasts
df_spain_forecast['MONTH_YEAR'] = df_spain_forecast['YEAR'].astype(str) + '-' + df_spain_forecast['MONTH'].astype(str).str.zfill(2)
df_spain_actuals['MONTH_YEAR'] = df_spain_actuals['YEAR'].astype(str) + '-' + df_spain_actuals['MONTH'].astype(str).str.zfill(2)


<ipython-input-143-4436db7fb944>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-143-4436db7fb944>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [144]:
df_spain_forecast.head()

,index,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT,FORECAST_MONTH,MONTH_DIFF,MONTH_YEAR
2,2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847,8,4.0,2023-12
20,20,Spain,Pepsi Regular (L3),2023,12,AI_forecast,AI_P05F,2023.0,134268.151080,4,8.0,2023-12
25,25,Spain,Lipton (L3),2025,3,AI_forecast,AI_P11F,2023.0,9702.217953,10,17.0,2025-03
62,62,Spain,7up Free (L3),2024,1,AI_forecast,AI_P04F,2023.0,70144.329753,3,10.0,2024-01
68,68,Spain,7up (L3),2024,4,AI_forecast,AI_P02F,2023.0,38882.921227,1,15.0,2024-04


In [153]:
import plotly.express as px

# Crear un gráfico de dispersión para df_spain_forecast, coloreado por MONTH_DIFF
fig = px.scatter(df_spain_forecast,
                 x='MONTH_YEAR',  # El eje x será la columna MONTH_YEAR (Mes-Año)
                 y='AMOUNT',      # El eje y será la cantidad de ventas (AMOUNT)
                 title='Ventas Pronosticadas a lo Largo del Tiempo (España)',  # Título del gráfico
                 labels={'MONTH_YEAR': 'Mes-Año', 'AMOUNT': 'Cantidad de Ventas'},  # Etiquetas de los ejes
                 color='MONTH_DIFF',  # Colorear por la columna MONTH_DIFF para mostrar cuán antigua es la predicción
                 color_continuous_scale='Viridis',  # Escala de colores para valores continuos
                 range_color=[1,18]  # Limitar la escala de colores entre 1 y 18 para la distancia de predicción
                 )  # Tema oscuro para contrastar mejor los colores

# Modificar la leyenda y título
fig.update_layout(
    title_x=0.5,  # Centrar el título
    coloraxis_colorbar=dict(title=dict(text='Distancia de predicción en años', side='right')), # Changed: moved standoff to the correct location
)

# Mostrar el gráfico
fig.show()



In [155]:
import plotly.express as px

# Definir el subbrand y el mes de diferencia que nos interesa
subbrand_of_interest = 'Pepsi Max (L3)'  # Ejemplo de subbrand
month_diff_of_interest = 3  # Ejemplo de mes de diferencia

# Filtrar los datos de forecast y de actual para ese subbrand y MONTH_DIFF
df_filtered_actuals = df_spain_actuals[
    (df_spain_actuals['SUBBRAND'] == subbrand_of_interest) ]

df_filtered_forecasts = df_spain_forecast[
    (df_spain_forecast['SUBBRAND'] == subbrand_of_interest) &
    (df_spain_forecast['MONTH_DIFF'] == month_diff_of_interest)
]

# Crear el gráfico de dispersión para las predicciones y valores reales
fig = px.scatter()

# Añadir los puntos de las ventas reales
fig.add_scatter(
    x=df_filtered_actuals['MONTH_YEAR'],  # Mes-Año
    y=df_filtered_actuals['AMOUNT'],  # Ventas reales
    mode='markers',  # Usar marcadores para las ventas reales
    name='Ventas Reales',
    marker=dict(color='blue', size=10)  # Color azul para las ventas reales
)

# Añadir los puntos de las predicciones
fig.add_scatter(
    x=df_filtered_forecasts['MONTH_YEAR'],  # Mes-Año
    y=df_filtered_forecasts['AMOUNT'],  # Ventas previstas
    mode='markers',  # Usar marcadores para las predicciones
    name='Predicciones',
    marker=dict(color='red', size=10)  # Color azul para las ventas reales
)

# Actualizar el diseño del gráfico
fig.update_layout(
    title=f'Comparación de Ventas Reales y Predicciones para {subbrand_of_interest} (Diferencia de Mes: {month_diff_of_interest})',
    xaxis_title='Mes-Año',
    yaxis_title='Ventas (Cantidad)',
    title_x=0.5,  # Centrar el título
)

# Mostrar el gráfico
fig.show()


In [156]:
# Definir el subbrand y el mes de diferencia que nos interesa
subbrand_of_interest = 'Pepsi Regular (L3)'  # Ejemplo de subbrand
month_diff_of_interest = 6  # Ejemplo de mes de diferencia

# Filtrar los datos de forecast y de actual para ese subbrand y MONTH_DIFF
df_filtered_actuals = df_spain_actuals[
    (df_spain_actuals['SUBBRAND'] == subbrand_of_interest) ]

df_filtered_forecasts = df_spain_forecast[
    (df_spain_forecast['SUBBRAND'] == subbrand_of_interest) &
    (df_spain_forecast['MONTH_DIFF'] == month_diff_of_interest)
]

# Crear el gráfico de dispersión para las predicciones y valores reales
fig = px.scatter()

# Añadir los puntos de las ventas reales
fig.add_scatter(
    x=df_filtered_actuals['MONTH_YEAR'],  # Mes-Año
    y=df_filtered_actuals['AMOUNT'],  # Ventas reales
    mode='markers',  # Usar marcadores para las ventas reales
    name='Ventas Reales',
    marker=dict(color='blue', size=10)  # Color azul para las ventas reales
)

# Añadir los puntos de las predicciones
fig.add_scatter(
    x=df_filtered_forecasts['MONTH_YEAR'],  # Mes-Año
    y=df_filtered_forecasts['AMOUNT'],  # Ventas previstas
    mode='markers',  # Usar marcadores para las predicciones
    name='Predicciones',
    marker=dict(color='red', size=10)  # Color azul para las ventas reales
)

# Actualizar el diseño del gráfico
fig.update_layout(
    title=f'Comparación de Ventas Reales y Predicciones para {subbrand_of_interest} (Diferencia de Mes: {month_diff_of_interest})',
    xaxis_title='Mes-Año',
    yaxis_title='Ventas (Cantidad)',
    title_x=0.5,  # Centrar el título
)

# Mostrar el gráfico
fig.show()

Conlusión: Viendo las distintas marcas y las predicciones según como de viejas eran, podemos observar que las predicciones son relativamente malas.